# Gera pdfs

## Carrega Pacotes

In [4]:
# Install a conda package in the current Jupyter kernel
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas

     |████████████████████████████████| 10.2MB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 15.1MB 29.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 29.1MB/s eta 0:00:01


In [5]:
import pandas as pd
import re
from titlecase import titlecase

## Define Constantes

In [45]:

strlatex = """
  \\documentclass[twoside]{{article}}
  \\usepackage[affil-it]{{authblk}}
  \\usepackage{{lipsum}} % Package to generate dummy text throughout \
this template
  \\usepackage{{eurosym}}
  \\usepackage[sc]{{mathpazo}} % Use the Palatino font
  \\usepackage[T1]{{fontenc}} % Use 8-bit encoding that has 256 glyphs
  \\usepackage[utf8]{{inputenc}}
  \\linespread{{1.05}} % Line spacing-Palatino needs more space between \
lines
  \\usepackage{{microtype}} % Slightly tweak font spacing for \
aesthetics\[IndentingNewLine]
  \\usepackage[hmarginratio=1:1,top=32mm,columnsep=20pt]{{geometry}} % \
Document margins
  \\usepackage{{multicol}} % Used for the two-column layout of the \
document
  \\usepackage[hang,small,labelfont=bf,up,textfont=it,up]{{caption}} % \
Custom captions under//above floats in tables or figures
  \\usepackage{{booktabs}} % Horizontal rules in tables
  \\usepackage{{float}} % Required for tables and figures in the \
multi-column environment-they need to be placed in specific locations \
with the[H] (e.g. \\begin{{table}}[H])
  \\usepackage{{hyperref}} % For hyperlinks in the PDF
  \\usepackage{{lettrine}} % The lettrine is the first enlarged letter \
at the beginning of the text
  \\usepackage{{paralist}} % Used for the compactitem environment which \
makes bullet points with less space between them
  \\usepackage{{abstract}} % Allows abstract customization
  \\renewcommand{{\\abstractnamefont}}{{\\normalfont\\bfseries}} 
  %\\renewcommand{{\\abstracttextfont}}{{\\normalfont\\small\\itshape}} % \
Set the abstract itself to small italic text\[IndentingNewLine]
  \\usepackage{{titlesec}} % Allows customization of titles
  \\renewcommand\\thesection{{\\Roman{{section}}}} % Roman numerals for \
the sections
  \\renewcommand\\thesubsection{{\\Roman{{subsection}}}} % Roman numerals \
for subsections
  \\titleformat{{\\section}}[block]{{\\large\\scshape\\centering}}{{\\\
thesection.}}{{1em}}{{}} % Change the look of the section titles
  \\titleformat{{\\subsection}}[block]{{\\large}}{{\\thesubsection.}}{{1em}}{{}}\
 % Change the look of the section titles
  \\usepackage{{fancyhdr}} % Headers and footers
  \\pagestyle{{fancy}} % All pages have headers and footers
  \\fancyhead{{}} % Blank out the default header
  \\fancyfoot{{}} % Blank out the default footer
  \\fancyhead[C]{{X-meeting $\\bullet$ October 2019 $\\bullet$ Campos do  \
Jord\\~ao}} % Custom header text
  \\fancyfoot[RO,LE]{{}} % Custom footer text
  %----------------------------------------------------------------------------------------
\
  % TITLE SECTION
  %\
----------------------------------------------------------------------------------------\
 
 
 \\title{{\\vspace{{-15mm}}\\fontsize{{24pt}}{{10pt}}\\selectfont\\textbf{{ {} }}}} % Article title
  
  
  \\author{{ {} }}
  
  \\affil{{ {} }}
  \\vspace{{-5mm}}
  \\date{{}}
  
  %----------------------------------------------------------------------------------------\
 
  
  \\begin{{document}}
  
  
  \\maketitle % Insert title
  
  
  \\thispagestyle{{fancy}} % All pages have headers and footers\

  %----------------------------------------------------------------------------------------\
  
  % ABSTRACT
  
  %----------------------------------------------------------------------------------------\
  
  
  \\begin{{abstract}}
  {}
  
  Funding: {} \\\\ 
  \\end{{abstract}}
  \\end{{document}} """

strlatex1 = """
  \\documentclass[twoside]{{article}}
  \\usepackage[affil-it]{{authblk}}
  \\usepackage{{lipsum}} % Package to generate dummy text throughout \
this template
  \\usepackage{{eurosym}}
  \\usepackage[sc]{{mathpazo}} % Use the Palatino font
  \\usepackage[T1]{{fontenc}} % Use 8-bit encoding that has 256 glyphs
  \\usepackage[utf8]{{inputenc}}
  \\linespread{{1.05}} % Line spacing-Palatino needs more space between \
lines
  \\usepackage{{microtype}} % Slightly tweak font spacing for \
aesthetics\[IndentingNewLine]
  \\usepackage[hmarginratio=1:1,top=32mm,columnsep=20pt]{{geometry}} % \
Document margins
  \\usepackage{{multicol}} % Used for the two-column layout of the \
document
  \\usepackage[hang,small,labelfont=bf,up,textfont=it,up]{{caption}} % \
Custom captions under//above floats in tables or figures
  \\usepackage{{booktabs}} % Horizontal rules in tables
  \\usepackage{{float}} % Required for tables and figures in the \
multi-column environment-they need to be placed in specific locations \
with the[H] (e.g. \\begin{{table}}[H])
  \\usepackage{{hyperref}} % For hyperlinks in the PDF
  \\usepackage{{lettrine}} % The lettrine is the first enlarged letter \
at the beginning of the text
  \\usepackage{{paralist}} % Used for the compactitem environment which \
makes bullet points with less space between them
  \\usepackage{{abstract}} % Allows abstract customization
  \\renewcommand{{\\abstractnamefont}}{{\\normalfont\\bfseries}} 
  %\\renewcommand{{\\abstracttextfont}}{{\\normalfont\\small\\itshape}} % \
Set the abstract itself to small italic text\[IndentingNewLine]
  \\usepackage{{titlesec}} % Allows customization of titles
  \\renewcommand\\thesection{{\\Roman{{section}}}} % Roman numerals for \
the sections
  \\renewcommand\\thesubsection{{\\Roman{{subsection}}}} % Roman numerals \
for subsections
  \\titleformat{{\\section}}[block]{{\\large\\scshape\\centering}}{{\\\
thesection.}}{{1em}}{{}} % Change the look of the section titles
  \\titleformat{{\\subsection}}[block]{{\\large}}{{\\thesubsection.}}{{1em}}{{}}\
 % Change the look of the section titles
  \\usepackage{{fancyhdr}} % Headers and footers
  \\pagestyle{{fancy}} % All pages have headers and footers
  \\fancyhead{{}} % Blank out the default header
  \\fancyfoot{{}} % Blank out the default footer
  \\fancyhead[C]{{X-meeting $\\bullet$ October 2019 $\\bullet$ Campos do  \
Jord\\~ao}} % Custom header text
  \\fancyfoot[RO,LE]{{}} % Custom footer text
  %----------------------------------------------------------------------------------------
\
  % TITLE SECTION
  %\
----------------------------------------------------------------------------------------\
 
 
 \\title{{\\vspace{{-15mm}}\\fontsize{{24pt}}{{10pt}}\\selectfont\\textbf{{ {} }}}} % Article title
  
  
  \\author{{ {} }}
  
  \\affil{{ {} }}
  \\vspace{{-5mm}}
  \\date{{}}
  
  %----------------------------------------------------------------------------------------\
 
  
  \\begin{{document}}
  
  
  \\maketitle % Insert title
  
  
  \\thispagestyle{{fancy}} % All pages have headers and footers\

  %----------------------------------------------------------------------------------------\
  
  % ABSTRACT
  
  %----------------------------------------------------------------------------------------\
  
  
  \\end{{document}} """


strlatex2 = """\\procpaper[switch=45,
    title={{{}}}, 
    author={{{}}}, 
    index={{{{{}}}}}]
    {{{}}} \n\n """

replacements={"Î¦" : "$\\Phi$ ", "\'" : "'", "ﬁ" : "fi", "á" : "\\'a",
 "\[Gamma]" : "$\\gamma$", "ï\[Not]<81>" : "fi",
 "\[RightArrow]" : "$\\rightarrow$", " Î\.b3" : "$ \\gamma$",
 " â<86><92> â<88><9e>" : " $\\rightarrow$", "â<80>\.b2" : "'",
 "Â\[PlusMinus]" : "$\\pm$", "â<89>¤" : "$\\leq$",
 "Â\[Degree]" :  "$^o$", "\Âº" : "$^o$", " \â<84>¢ CL" : " ",
 "Â\.b4" : "'", "â<89>¥" : "$\\leq$", "â<80><8b>â" : " ", "¹" : "1",
 "â" : "\\^a", "\[OpenCurlyDoubleQuote]" : "``",
 "\[CloseCurlyDoubleQuote]" : "''", "ã" : "\\~a", "á" : "\\'a",
 "ê" : "\\^e", "é" : "\\'e", "õ" : "\\~o", "ó" : "\\'o",
 "ô" : "\\^o", "ú" : "\\'u", "ç" : "\\c{c}", "í" : "\\'{\\i}",
 "Ã" : "\\~A", "Á" : "\\'A", "Ê" : "\\^E", "É" : "\\'E",
 "Õ" : "\\~O", "Ó" : "\\'O", "Ô" : "\\^O", "Ú" : "\\'U",
 "Ç" : "\\c{C}", "Í" : "\\'I", "Å" : "$\\AA$", "ß" : "$\\beta$",
 "\[CloseCurlyQuote]" : "'", "\[OpenCurlyQuote]" : "'", "%" : "\\%",
  "ß" : "$\\beta$",  "_" : "\\_",         
              "±":"$\pm$",
              "´":"", "µ":"$\mu$",
              ",":", ",
              "&":"\&",
 "ö" : "\\\"o", "ü" : "\\\"u",
 "à" : "\\`a", "À" : "\\`A", "€": "\\euro",
 "®" : "\\textsuperscript{\\textcopyright}", "¾" : "3/4",
 "ñ" : "\\~n", "ï" : "\\\"{\\i}", "I'" : "\\'I", "#" : "\\#",
 "i\.b4" : "\\'{\\i}",
 " " : " ",  "°" : "$^o$"}
prefix=""

## Funções Gerais

In [7]:
def namecase(str):
    """Converte Strings para Capitalização Brasileira"""
    lissubs=[(" De "," de "),(" Do "," do "), (" Da ", " da "), (" Das ", " das "),
             (" Dos "," dos ")]
    str2=titlecase(str)
    for subs in lissubs:
        str2=str2.replace(subs[0],subs[1])
    return str2



In [8]:
def brnames(str):
    """Gera Nomes para a regra ABNT"""
    lis=str.split(" ")
    str2=lis[-1]+ ", "
    for word in lis[0:-1]:
        str2=str2+" "+word
    return str2

In [9]:
def orderedunion(lis):
    """Elimina Repetições e mantém a ordem"""
    lis2=[]
    for elem in lis:
        if not (elem in lis2):
            lis2.append(elem)
    return lis2

In [10]:


def fixbin(string,replacements):
    for k, v in replacements.items():
        string=string.replace(k, v)
    return string

## Funções Específicas


In [11]:
def geraAuthors(str):
    """Gera uma Lista de Autores"""
    m = re.findall('[^(),]+\([^()]+[^()]+\)', str)
    return [namecase(word.split("(")[0].strip()) for word in m]

def geraAfil(str):
    """Gera uma lista de Afiliações"""
    m = re.findall('\([^()]+[^()]+\)', str)
    return [inst[1:-1] for inst in m]

In [12]:
artsds["Autores"]

NameError: name 'artsds' is not defined

In [13]:
def geraStringAfilOld(lis):
    lisafil=orderedunion(lis)
    stringafil="" 
    i=1
    for inst in lisafil:
        stringafil=stringafil+str(i)+" "+inst+"\n\n"
        i=i+1
    return stringafil

def geraStringAfil(lis):
    return fixbin(record["Institution"],replacements)
    
def geraStringAuthorsOld(record):
    lisafil=record["Afil"]
    lisauthors=record["Authors"]
    dictauthors={}
    dictafil={}
    lisafilorder=orderedunion(lisafil)
    for i in range(len(lisauthors)):
        dictauthors[lisauthors[i]]=lisafil[i]
    for i in range(len(lisafilorder)):
        dictafil[lisafilorder[i]]=i
    strsaida=""
    for author in lisauthors:
        num=dictafil[dictauthors[author]]
        strsaida=strsaida+author+"$^{"+str(num+1)+"}$, "
    return strsaida[:-1]

def geraStringAuthors(record):
    return record["Autores"]

    

## Entrada dos Dados

In [23]:
!ls

Relatorio de submissoes avaliadas_14-10-2019_14-58-12.xlsx
Relatorio_de_submissoes_avaliadas_24-10-2019.xlsx
auto
compila.bash
confprocxmeeting.cls
gerapdf.ipynb
papers
papers.log
papers.tex
posters.log
posters.tex
posters2.tex
proctext.log
proctext.tex
procxmeeting2017.aux
procxmeeting2017.idx
procxmeeting2017.ilg
procxmeeting2017.ind
procxmeeting2017.log
procxmeeting2017.npc}
procxmeeting2017.nps
procxmeeting2017.nps}
procxmeeting2017.pdf
procxmeeting2017.tex
procxmeeting2017.toc
procxmeeting2019.aux
procxmeeting2019.idx
procxmeeting2019.ilg
procxmeeting2019.ind
procxmeeting2019.log
procxmeeting2019.nps
procxmeeting2019.pdf
procxmeeting2019.tex
procxmeeting2019.toc


In [34]:
artsds=pd.read_excel(prefix+'Relatorio_de_submissoes_avaliadas_24-10-2019.xlsx',index_col=0)

In [35]:
artsds

,Codigo,Modalidade,Área Temática,Título,Autores,Emails,Apresentadores,Situação,Apresentado,Submeteu Trabalho Completo,Resumo,Link Trabalho Completo,Arquivo,Arquivo do Resumo,Funding,Institution
Número,,,,,,,,,,,,,,,,
198424,DA,Dissertation Award,Database and Software Development,N3O: A NEAT expansion for improving classifica...,"Bruno Iochins Grisci,Marcio Dorn,Mario Inostro...","bigrisci@inf.ufrgs.br,mdorn@inf.ufrgs.br,mario...",Bruno Iochins Grisci,Aprovado,Não,Não,Microarrays are one of the primary techniques ...,NaN,NaN,https://even3.blob.core.windows.net/formulario...,NaN,Universidade Federal do Rio Grande do Sul (UFRGS)
201719,DA,Dissertation Award,RNA and Transcriptomics,Uncovering the mouse olfactory long non-coding...,Antônio Pedro de Castello Branco da Rocha Cama...,"antoniop.camargo@gmail.com,t.seike@gmail.com,m...",Antônio Pedro de Castello Branco da Rocha Camargo,Aprovado,Não,Não,(PDF formatado e com figuras em anexo)\n\nAbst...,NaN,NaN,https://even3.blob.core.windows.net/formulario...,NaN,Universidade Estadual de Campinas
190947,DA01,Dissertation Award,Proteins and Proteomics,DETECÇÃO E VISUALIZAÇÃO DE SUBESTRUTURAS COMUN...,"Vagner Soares Ribeiro,Charles A. Santana,Alexa...","vsheik@gmail.com,charles.santana@ufv.br,alexan...",Vagner Soares Ribeiro,Aprovado,Não,Não,O volume de dados disponíveis sobre sequências...,NaN,NaN,https://even3.blob.core.windows.net/formulario...,NaN,"Universidade Federal de Viçosa, UFV"
190830,HT,HighLight Tracks,Database and Software Development,mirtronDB: a mirtron knowledge base,"Bruno Henrique Ribeiro da Fonseca,Douglas Silv...","fonseca.adm@hotmail.com,doug@rc.unesp.br,pasch...",Alexandre R Paschoal,Aprovado,Não,Não,Mirtrons arise from short introns with atypica...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,UTFPR
188974,HT,HighLight Tracks,Genes and Genomics,Genomic analysis unveils important aspects of ...,"Hemanoel Passarelli Araujo,Jussara Kasuko Palm...","hemanuel.passarelli@gmail.com,jukasuko@gmail.c...",Hemanoel Passarelli Araujo,Aprovado,Não,Não,Klebsiella aerogenes is an important pathogen ...,NaN,https://even3.blob.core.windows.net/formulario...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202192,TR31,Poster,RNA and Transcriptomics,piRNAs expression profiles: Estimates and insi...,"Ricardo Piuco,Pedro A F Galante","rpiuco@mochsl.org.br,pgalante@gmail.com",Ricardo Piuco,Aprovado,Não,Não,Small RNAs plays several roles regulating gene...,NaN,NaN,NaN,NaN,NaN
201518,TR32,Poster,RNA and Transcriptomics,Circular RNAs contribute to tumorigenesis and ...,"Vanessa Galdeno Freitas,Pedro Alexandre Favore...","vanessagaldeno@gmail.com,pedro.afgalante@hsl.o...",Vanessa Galdeno Freitas,Aprovado,Não,Não,Circular RNAs (circRNAs) are a new class of RN...,NaN,NaN,NaN,CAPES,Interunidades em Bioinformática - IME USP / In...
190102,TR33,Poster,RNA and Transcriptomics,The effect of genetic diversity in differentia...,"Victor Mello,Ana Letycia Basso Garcia,Fernando...","victor.pessoa@usp.br,garcia.alb@usp.br,fernand...",Victor Mello,Aprovado,Não,Não,Differential gene expression studies focus on ...,NaN,NaN,NaN,FAPESP,ESALQ - USP


Insere duas colunas no lugar da Coluna de Autores

In [36]:
artsds.columns

Index(['Codigo', 'Modalidade', 'Área Temática', 'Título', 'Autores', 'Emails',
       'Apresentadores', 'Situação', 'Apresentado',
       'Submeteu Trabalho Completo', 'Resumo', 'Link Trabalho Completo',
       'Arquivo', 'Arquivo do Resumo', 'Funding', 'Institution'],
      dtype='object')

In [37]:
artsds["Autores"]

Número
198424    Bruno Iochins Grisci,Marcio Dorn,Mario Inostro...
201719    Antônio Pedro de Castello Branco da Rocha Cama...
190947    Vagner Soares Ribeiro,Charles A. Santana,Alexa...
190830    Bruno Henrique Ribeiro da Fonseca,Douglas Silv...
188974    Hemanoel Passarelli Araujo,Jussara Kasuko Palm...
                                ...                        
202192                      Ricardo Piuco,Pedro A F Galante
201518    Vanessa Galdeno Freitas,Pedro Alexandre Favore...
190102    Victor Mello,Ana Letycia Basso Garcia,Fernando...
201385    Vinicius da Silva Coutinho Parreira,Letícia Gr...
196669    Thiago Castilho Elias,William Mesquita da Cost...
Name: Autores, Length: 178, dtype: object

## Separação em Grupos

In [47]:
artsdsgrouped=artsds.groupby(["Modalidade"],axis=0)
artsdsposter=artsdsgrouped.get_group('Poster')
artsdsHT=artsdsgrouped.get_group('HighLight Tracks')
artsdsDT=artsdsgrouped.get_group('Dissertation Award')


In [48]:
artsds.iloc[0].index

Index(['Codigo', 'Modalidade', 'Área Temática', 'Título', 'Autores', 'Emails',
       'Apresentadores', 'Situação', 'Apresentado',
       'Submeteu Trabalho Completo', 'Resumo', 'Link Trabalho Completo',
       'Arquivo', 'Arquivo do Resumo', 'Funding', 'Institution'],
      dtype='object')

## Gera os arquivos com os abstracts

In [49]:
for index, record in artsds.iterrows():
    #print(record["Autores"])
    author=fixbin(record["Autores"],replacements)
    title=fixbin(record["Título"],replacements)
    if record["Institution"]!=record["Institution"]:
        afiliation=""
    else:
        afiliation=fixbin(record["Institution"],replacements)
    print(index)
    if record["Modalidade"]=="Poster":
        abstract=fixbin(record["Resumo"],replacements)
    else:
        abstract=""
    if record["Funding"]!=record["Funding"]:
        funding=""
    else:
        funding=fixbin(record["Funding"],replacements)
    if record["Modalidade"]=="Poster":
        stringfileresumo=strlatex.format(title,author,afiliation,abstract,funding)
    else:
        stringfileresumo=strlatex1.format(title,author,afiliation)
    f = open('papers/art'+str(index)+".tex", 'w')
    f.write(stringfileresumo)
    f.close()

198424
201719
190947
190830
188974
201371
190030
190542
180929
201545
190044
211733
189628
189566
189373
190043
190045
201623
211385
211699
201254
188839
210676
201343
201421
201610
211863
208811
198393
190251
191455
201613
201674
201447
201317
201202
190847
210683
181353
202805
201465
190060
192871
201712
201730
201344
202045
201829
211561
193434
199174
211838
190658
201327
201568
200200
201354
211308
195426
202006
189638
211276
212533
212539
211575
201722
201636
196221
211208
201634
201635
201585
197514
201328
211358
211386
201444
211524
201772
211728
199542
201643
201645
211217
212001
212187
194642
201650
201638
201185
201350
201584
180792
208813
211986
201507
202076
210930
211177
201630
201671
200863
211423
210611
201860
190910
201223
196113
190006
190850
211375
211380
211193
193435
199094
209321
201380
202000
202003
202028
202034
201942
191535
201699
201740
201494
201728
201454
198401
201660
200803
201734
190288
186188
201655
190633
201661
200294
201515
200819
199175
200264
201276

In [50]:
!pwd

/Users/lemke/Documents/procxmeeting2019


## Gera os arquivos com os índices 

In [51]:
areas = sorted(list(set(artsdsposter["Área Temática"])))
groupareas=artsdsposter.groupby(["Área Temática"],axis=0)
compstr = "";
for area in areas:
    print(area)
    ds=groupareas.get_group(area)
#    ds=ds.sort_values("Poster Code")
    sessionstr = "\\chapter{" + area + "}\n";
    for index, record in ds.iterrows():
        title=fixbin(record["Título"],replacements)
        author=record["Autores"]
        file="art"+str(index)
        indexstr=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Autores"].split(",")])
        sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
    compstr = compstr + sessionstr
f = open('posters.tex', 'w')
f.write(compstr)
f.close()

Database and Software Development
Genes and Genomics
Phylogeny and Evolution
Proteins and Proteomics
RNA and Transcriptomics
Systems Biology and Networks


In [52]:
compstr = "";
ds=artsdsHT
sessionstr=""
#ds=ds.sort_values("Poster Code")
for index, record in ds.iterrows():
    print(index)
    title=fixbin(record["Título"],replacements)
    author=record["Autores"]
    file="art"+str(index)
    indexstr=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Autores"].split(",")])
    sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
compstr = compstr + sessionstr
f = open('papers.tex', 'w')
f.write(compstr)
f.close()

190830
188974
201371
190030
190542
180929
201545


In [53]:
compstr = "";
ds=artsdsDT
sessionstr=""
#ds=ds.sort_values("Poster Code")
for index, record in ds.iterrows():
    print(index)
    title=fixbin(record["Título"],replacements)
    author=record["Autores"]
    file="art"+str(index)
    indexstr=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Autores"].split(",")])
    sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
compstr = compstr + sessionstr
f = open('teses.tex', 'w')
f.write(compstr)
f.close()

198424
201719
190947
